In [1]:
pip install PyMuPDF pdfplumber pandas sentence_transformers chromadb openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 106.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 107.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 95.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 80.8 MB/s eta 0:00:

In [2]:
import os
import re
import fitz
import pdfplumber
from PIL import Image
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from transformers import CLIPProcessor, CLIPModel
from openai import OpenAI
from collections import Counter

In [3]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
import os
import re
import fitz
import pdfplumber
from PIL import Image
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from transformers import CLIPProcessor, CLIPModel
from openai import OpenAI
from collections import Counter


In [ ]:
def fallback_save_full_page(doc, page_num, scale, output_path):
    """
    Save the entire page as an image if cropping fails.
    """
    page = doc.load_page(page_num - 1)
    pix = page.get_pixmap(matrix=fitz.Matrix(scale, scale))
    img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
    img.save(output_path)
    return img

In [ ]:
def extract_table(plumber_page, doc, page_num, output_path, dpi):
    """
    Crop the largest table on the page; fallback to full-page save if needed.
    """
    tables = plumber_page.find_tables()
    if tables:
        table = max(tables, key=lambda t: (t.bbox[2] - t.bbox[0]) * (t.bbox[3] - t.bbox[1]))
        x0 = max(0, table.bbox[0] - 30)
        y0 = max(0, table.bbox[1] - 30)
        x1 = min(plumber_page.width, table.bbox[2] + 30)
        y1 = min(plumber_page.height, table.bbox[3] + 5)
        if x1 > x0 and y1 > y0:
            img = plumber_page.crop((x0, y0, x1, y1)).to_image(resolution=dpi)
            img.save(output_path)
            return img
    return fallback_save_full_page(doc, page_num, 2, output_path)


In [ ]:
def get_figure_clip_rect(page, fig_id):
    """
    Determine the clipping rectangle and crop type for a given figure ID.
    """
    width, height = page.rect.width, page.rect.height
    clip = fitz.Rect(20, 20, width - 20, height - 20)
    crop_type = "full page fallback"

    if fig_id in {"1-1", "1-2", "1-3"}:
        end_y = height * 0.85
        for b in page.get_text("blocks"):
            if f"figure {fig_id}" in b[4].lower():
                end_y = b[1] - 10
                break
        clip = fitz.Rect(40, 20, width - 40, end_y)
        crop_type = f"full figure from top for {fig_id}"
    elif fig_id == "2-1":
        clip = fitz.Rect(0, height * 0.55, width, height)
        crop_type = "center to bottom"
    elif fig_id == "2-2":
        clip = fitz.Rect(0, height * 0.52, width, height)
        crop_type = "center to bottom"
    elif fig_id == "2-3":
        clip = fitz.Rect(0, 0, width, height * 0.45)
        crop_type = "top 45%"
    elif fig_id == "2-4":
        clip = fitz.Rect(0, 0, width, height * 0.3)
        crop_type = "top 30%"
    elif fig_id in {"2-5", "2-6", "2-7"}:
        clip = fitz.Rect(0, 0, width, height * 0.4)
        crop_type = "top 40%"
    elif fig_id == "1-6":
        clip = fitz.Rect(0, height * 0.57, width, height * 0.95)
        crop_type = "57% to 95%"
    elif fig_id == "1-4":
        clip = fitz.Rect(0, height * 0.5, width, height * 0.95)
        crop_type = "50% to 95%"
    elif fig_id == "1-5":
        clip = fitz.Rect(20, 20, width - 20, height - 20)
        crop_type = "full page with margins"
    else:
        for b in page.get_text("blocks"):
            text = b[4].lower()
            if f"figure {fig_id}" in text or f"figure {fig_id.replace('-', '.')}" in text:
                clip = fitz.Rect(40, 50, width - 40, b[1] - 10)
                crop_type = "above caption"
                break
    return clip, crop_type

In [ ]:
def generate_search_terms(content_id, content_type):
    """
    Build search patterns for captions of figures and tables.
    """
    if content_type == "figure":
        fig_id = content_id.replace("Figure_", "")
        dot = fig_id.replace("-", ".")
        variants = [
            f"Figure {fig_id}", f"Figure {dot}",
            f"Fig. {fig_id}", f"Fig. {dot}"
        ]
    else:
        table_id = content_id.replace("Table_", "")
        dot = table_id.replace("_", ".")
        variants = [f"Table {table_id}", f"Table {dot}"]
    return [v.lower() for v in variants]

In [ ]:
def extract_captions(doc, content_locations):
    """
    Extract captions for each figure and table using generated search terms.
    """
    data = {}
    for cid, page_num, ctype in content_locations:
        try:
            page = doc.load_page(page_num - 1)
            blocks = page.get_text("blocks")
            terms = generate_search_terms(cid, ctype)
            caption = next((blk[4].strip() for blk in blocks if any(t in blk[4].lower() for t in terms)), None)
            if caption:
                data[cid] = caption
        except Exception as e:
            print(f"Error extracting caption for {cid}: {e}")
    return data

In [ ]:
def extract_content(pdf_path, output_folder="extracted_content"):
    """
    Extract structured images (tables and figures) from a PDF.
    """
    os.makedirs(output_folder, exist_ok=True)
    doc = fitz.open(pdf_path)
    plumber = pdfplumber.open(pdf_path)

    content_locations = [
        ("Table_1_1", 1, "table"), ("Figure_1-1", 2, "figure"),
        ("Figure_1-2", 3, "figure"), ("Figure_1-3", 4, "figure"),
        ("Table_5_1", 5, "table"), ("Figure_1-4", 5, "figure"),
        ("Table_7_1", 7, "table"), ("Figure_1-5", 8, "figure"),
        ("Figure_1-6", 10, "figure"), ("Table_11_1", 11, "table"),
        ("Figure_2-1", 20, "figure"), ("Figure_2-2", 21, "figure"),
        ("Figure_2-3", 24, "figure"), ("Figure_2-4", 27, "figure"),
        ("Figure_2-5", 29, "figure"), ("Figure_2-6", 30, "figure"),
        ("Figure_2-7", 32, "figure")
    ]
    captions = extract_captions(doc, content_locations)
    extracted = {"tables": {}, "figures": {}}
    reference_mapping = {}
    dpi = 300
    counter = 1

    for cid, page_num, ctype in content_locations:
        page = doc.load_page(page_num - 1)
        output_path = os.path.join(output_folder, f"{counter}.png")

        if ctype == "table":
            plumber_page = plumber.pages[page_num - 1]
            extract_table(plumber_page, doc, page_num, output_path, dpi)
            extracted["tables"][cid] = {
                "page": page_num,
                "path": output_path,
                "type": "table",
                "number": counter,
                "caption": captions.get(cid, ""),
                "original_id": cid
            }
        else:
            clip_rect, crop_type = get_figure_clip_rect(page, cid.replace("Figure_", ""))
            pix = page.get_pixmap(matrix=fitz.Matrix(2.5, 2.5), clip=clip_rect)
            Image.frombytes("RGB", [pix.width, pix.height], pix.samples).save(output_path)
            extracted["figures"][cid] = {
                "page": page_num,
                "path": output_path,
                "type": "figure",
                "number": counter,
                "crop_type": crop_type,
                "caption": captions.get(cid, ""),
                "original_id": cid
            }

        reference_mapping[cid] = counter
        counter += 1

    plumber.close()
    doc.close()
    return extracted, reference_mapping


In [ ]:
def sliding_window_chunks(words, chunk_size, overlap):
    step = max(1, chunk_size - overlap)
    for i in range(0, len(words), step):
        yield words[i:i + chunk_size]

In [5]:
def extract_and_chunk_text(pdf_path, chunk_size=100, overlap=20):
    doc = fitz.open(pdf_path)
    all_chunks = []
    for i, page in enumerate(doc, start=1):
        text = page.get_text()
        paragraphs = re.split(r"\n\s*\n", text)
        for para in paragraphs:
            para = para.strip()
            words = para.split()
            if len(words) > chunk_size:
                for win in sliding_window_chunks(words, chunk_size, overlap):
                    all_chunks.append({"page": i, "text": " ".join(win), "type": "text"})
            elif len(words) > 5:
                all_chunks.append({"page": i, "text": para, "type": "text"})
    doc.close()
    return all_chunks

In [ ]:
def setup_text_and_image_db(pdf_path, extracted, reference_mapping):
    text_chunks = extract_and_chunk_text(pdf_path)
    caption_chunks = []
    semantic_mapping = {}

    for ctype in ["tables", "figures"]:
        for cid, data in extracted[ctype].items():
            cap = data.get("caption")
            if cap:
                caption_chunks.append({
                    "page": data["page"],
                    "text": cap,
                    "type": "caption",
                    "image_number": data["number"],
                    "original_id": cid
                })
                terms = generate_search_terms(cid, ctype[:-1])
                for t in terms:
                    semantic_mapping[t] = data["number"]

    all_chunks = text_chunks + caption_chunks
    df = pd.DataFrame(all_chunks)

    # Text embeddings
    text_model = SentenceTransformer('intfloat/e5-large')
    df['embedding'] = df['text'].apply(lambda x: text_model.encode(x, normalize_embeddings=True))
    chroma_client = chromadb.PersistentClient(path="./chroma_db")
    text_collection = chroma_client.get_or_create_collection(name="text_embeddings", metadata={"hnsw:space": "cosine"})
    text_ids = [f"text_{i}" for i in df.index]
    text_embeds = [emb.tolist() for emb in df['embedding']]
    text_metas = []
    for _, row in df.iterrows():
    # Safely pull out image_number (default 0) and original_id (default "")
          img_val = row.get('image_number', 0)
          img_num = 0 if pd.isna(img_val) else int(img_val)

          orig_val = row.get('original_id', '')
          orig_id = '' if pd.isna(orig_val) else orig_val

          meta = {
              "page": int(row['page']),
              "text": row['text'],
              "type": row['type'],
              "image_number": img_num,
              "original_id": orig_id
          }
          text_metas.append(meta)
    text_collection.upsert(ids=text_ids, embeddings=text_embeds, metadatas=text_metas)

    # Image embeddings
    clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
    clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
    image_folder = "extracted_content"
    image_paths = sorted([os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.lower().endswith('.png')])
    image_embs, image_ids, image_metas = [], [], []
    for path in image_paths:
        img = Image.open(path).convert("RGB")
        inputs = clip_processor(images=img, return_tensors="pt")
        feats = clip_model.get_image_features(**inputs)
        emb = feats[0].detach().numpy()
        num = int(os.path.splitext(os.path.basename(path))[0])
        image_embs.append(emb)
        image_ids.append(f"image_{num}")
        cid = next((cid for cid, num2 in reference_mapping.items() if num2 == num), '')
        image_metas.append({"image_number": num, "content_id": cid})
    if image_embs:
        emb_mat = np.vstack(image_embs)
        image_collection = chroma_client.get_or_create_collection(name="image_embeddings", metadata={"hnsw:space": "cosine"})
        image_collection.upsert(ids=image_ids, embeddings=emb_mat.tolist(), metadatas=image_metas)
    else:
        image_collection = None

    return text_model, clip_model, clip_processor, text_collection, image_collection, semantic_mapping, reference_mapping

In [ ]:
def instrument_distances(questions, text_model, clip_model, clip_processor, text_collection, image_collection):
    caption_dists, clip_dists = [], []
    for q in questions:
        te = text_model.encode(q, normalize_embeddings=True)
        inp = clip_processor(text=[q[:480]], return_tensors="pt", padding=True, truncation=True, max_length=77)
        ce = clip_model.get_text_features(**inp)[0].detach().numpy()

        tr = text_collection.query(query_embeddings=[te.tolist()], n_results=5, include=["metadatas","distances"])
        md, ds = tr["metadatas"][0], tr["distances"][0]
        for m, d in zip(md, ds):
            if m["type"] == "caption":
                caption_dists.append(d)
                break

        if image_collection:
            ir = image_collection.query(query_embeddings=[ce.tolist()], n_results=3, include=["distances"])
            clip_dists.append(ir["distances"][0][0])

    return caption_dists, clip_dists

In [ ]:
def calibrate_thresholds(caption_dists, clip_dists, quantile=0.25):
    cap_thresh = float(np.quantile(caption_dists, quantile))
    clip_thresh = float(np.quantile(clip_dists, quantile))
    return cap_thresh, clip_thresh

In [ ]:
def retrieve_relevant_content(question, text_emb, clip_emb,
                              text_collection, image_collection,
                              semantic_mapping, reference_mapping,
                              cap_thresh, clip_thresh, k=25):
    tr = text_collection.query(query_embeddings=[text_emb.tolist()],
                               n_results=k, include=["metadatas","distances"])
    metas, dists = tr["metadatas"][0], tr["distances"][0]
    items = list(zip(metas, dists))
    texts = [(m, 1/(d+1e-6)) for m, d in items if m["type"] == "text"]
    texts.sort(key=lambda x: x[1], reverse=True)
    context = " ".join(m[0]["text"] for m in texts[:10])

    # 1) regex
    m = re.search(r"\b(?:fig(?:ure)?|table)\s*\.?\s*(\d+)[\._-]?(\d+)\b", question, re.I)
    if m:
        kind = "figure" if m.group(0).lower().startswith("fig") else "table"
        num = f"{m.group(1)}-{m.group(2)}"; key = f"{kind} {num}".lower()
        img = semantic_mapping.get(key)
        if img:
            orig = next((cid for cid, n in reference_mapping.items() if n == img), "")
            return context, img, "regex_mention", orig

    # 2) caption
    caps = [(m, d) for m, d in items if m["type"] == "caption"]
    if caps:
        best_meta, best_dist = min(caps, key=lambda x: x[1])
        if best_dist <= cap_thresh:
            return context, best_meta["image_number"], "caption_match", best_meta.get("original_id", "")

    # 3) clip
    if image_collection:
        ir = image_collection.query(query_embeddings=[clip_emb.tolist()],
                                    n_results=3, include=["metadatas","distances"])
        cd = ir["distances"][0]
        if cd and cd[0] <= clip_thresh:
            meta = ir["metadatas"][0][0]
            return context, int(meta.get("image_number", 0)), "clip_match", meta.get("content_id", "")

    # 4) context mention (soft)
    ctx_lower = context.lower()
    for pat, img in semantic_mapping.items():
        if pat in ctx_lower:
            orig = next((cid for cid, n in reference_mapping.items() if n == img), "")
            return context, img, "context_mention", orig

    return context, 0, "no_match", ""

In [ ]:
def generate_openai_answer(client, question, context):
    instr = ("You are an economics professor. Using only the context, "
             "provide a clear, complete answer in full sentences.")
    prompt = f"{instr}\nQuestion: {question}\nContext: {context}\nAnswer:"
    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": instr},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2,
        max_tokens=150
    )
    ans = resp.choices[0].message.content.strip()
    return re.sub(r"\s+", " ", ans)

In [ ]:
def process_question(client, question, qid, tm, clip_model, clip_processor,
                     text_collection, image_collection, semantic_mapping, reference_mapping,
                     cap_thresh, clip_thresh):
    te = tm.encode(question, normalize_embeddings=True)
    inp = clip_processor(text=[question[:480]], return_tensors="pt", padding=True, truncation=True, max_length=77)
    ce = clip_model.get_text_features(**inp)[0].detach().numpy()
    context, img, reason, orig = retrieve_relevant_content(
        question, te, ce,
        text_collection, image_collection,
        semantic_mapping, reference_mapping,
        cap_thresh, clip_thresh
    )
    answer = generate_openai_answer(client, question, context)
    return {'ID': qid, 'Text': answer, 'Image': img}

In [6]:
def main():
    pdf_path = "document.pdf"
    extracted, ref_map = extract_content(pdf_path)
    tm, clip_model, clip_processor, text_collection, image_collection, semantic_mapping, reference_mapping = setup_text_and_image_db(
        pdf_path, extracted, ref_map
    )

    questions_df = pd.read_csv("Lab_2_Part_1_Questions.csv")
    questions = questions_df['Question'].tolist()

    cap_dists, clip_dists = instrument_distances(
        questions, tm, clip_model, clip_processor, text_collection, image_collection
    )
    cap_thresh, clip_thresh = calibrate_thresholds(cap_dists, clip_dists, quantile=0.25)

    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    results = []
    for _, row in questions_df.iterrows():
        results.append(process_question(
            client, row['Question'], row.get('ID'),
            tm, clip_model, clip_processor,
            text_collection, image_collection,
            semantic_mapping, reference_mapping,
            cap_thresh, clip_thresh
        ))

    out_df = pd.DataFrame(results)[['ID','Text','Image']]
    out_df.to_csv("submission.csv", index=False)
    print("Done. Thresholds:", cap_thresh, clip_thresh)

if __name__ == "__main__":
    main()


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

Done. Thresholds: 0.141426682472229 0.6803140044212341
